In [2]:
from pyciss.ringcube import RingCube
import pandas as pd
import json

# Read in classification CSV and expand JSON fields
classifications = pd.read_csv('10-image-sample-classifications.csv')  #<--*****Insert Filename Here<--
classifications['metadata_json'] = [json.loads(q) for q in classifications.metadata]
classifications['annotations_json'] = [json.loads(q) for q in classifications.annotations]
classifications['subject_data_json'] = [json.loads(q) for q in classifications.subject_data]

# Calculate number of markings per classification
# Note: index of annotations_json ("q" here) corresponds to task number (i.e., 0)
classifications['n_markings'] = [ len(q[0]['value']) for q in classifications.annotations_json ]

### Classification Selection / CURRENT SETTING: most recent workflow version
# OPTION 1: Select only classifications from most recent workflow version
iclass = classifications[classifications.workflow_version == classifications['workflow_version'].max()]

# OPTION 2: Select most/all valid classifications using workflow_id and workflow_version
#iclass = classifications[(classifications['workflow_id'] == 1687) & (classifications['workflow_version'] > 40)]

#/(1000.))*(radMax-radMin) + radMin
# Output markings from classifications in iclass to new list of dictionaries (prep for pandas dataframe)
# Applicable for workflows with marking task as first task, and outputs data for circular markers (x,y,r)
clist=[]
for index, c in iclass.iterrows():
    if c['n_markings'] > 0:
        # Note: index of annotations_json corresponds to task number (i.e., 0)
        for q in c.annotations_json[0]['value']:
            
            cube = RingCube(c.subject_data_json[str(c.subject_ids)]['image1'][:-8])
            radMin = cube.minrad.value
            radMax = cube.maxrad.value
            
            # OPTIONAL EXPANSION: could use if statement here to split marker types
            
            clist.append({'classification_id':c.classification_id, 'user_name':c.user_name,'subject_ids':c.subject_data_json[str(c.subject_ids)]['image1'][:-8], 'radius':(((q['x']/(1000.))*(radMax-radMin)) + radMin)})

# Output list of dictionaries to pandas dataframe and export to CSV.
col_order=['classification_id','user_name','subject_ids','radius']
out=pd.DataFrame(clist)[col_order]
#out.to_csv(markfile_out,index_label='mark_id')
out

,classification_id,user_name,subject_ids,radius
0,149171147,chab5881,N1595338491,132.212318
1,149171786,chab5881,N1467344745,96.344622
2,149171964,chab5881,N1625958972,134.297717
3,149172573,chab5881,N1467345680,125.377792
4,149172573,chab5881,N1467345680,125.447928
5,149189342,michaelaye,N1584974162,136.440427
6,149189342,michaelaye,N1584974162,136.703015
7,149189453,michaelaye,N1467344745,96.344817
8,149189691,michaelaye,N1467345680,125.285091
9,149189691,michaelaye,N1467345680,125.383994
